In [ ]:
from reaktoro import *
import numpy as np
import os

In [ ]:
results_folder = 'results-fixed-fugacity'
os.system('mkdir -p ' + results_folder)

In [ ]:
db = PhreeqcDatabase("phreeqc.dat")

In [ ]:
solution = AqueousPhase(speciate("H O C Na Cl Ca"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

In [ ]:
mineral = MineralPhase("Calcite")

In [ ]:
system = ChemicalSystem(db, solution, mineral)

In [ ]:
props = ChemicalProps(system)
aprops = AqueousProps(system)

In [ ]:
state = ChemicalState(system)
state.set("H2O", 1.0, "kg")
state.set("Calcite", 10.0, "mol")

In [ ]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2")

In [ ]:
solver = EquilibriumSolver(specs)

In [ ]:
conditions = EquilibriumConditions(specs)

In [ ]:
def equilibrate(T, ppCO2):

    conditions.temperature(T, "celsius")
    conditions.pressure(1.0, "atm")
    conditions.fugacity("CO2", 10**(ppCO2))

    solver.solve(state, conditions)

    props.update(state)
    aprops.update(state)
    pH = aprops.pH()[0]

    return pH

In [ ]:
num_temperatures = 1
num_ppressures = 101
temperatures = np.array([50])
#temperatures = np.flip(np.linspace(0, 50.0, num=num_temperatures))
co2ppressures = np.linspace(-4.0, 0.0, num=num_ppressures)

num_temperatures = 1 #101
num_ppressures = 1 #106
temperatures = np.array([50.0])
co2ppressures = np.array([-4.0])

In [ ]:
pHs = np.zeros((num_ppressures, num_temperatures))

In [ ]:
print(temperatures)
print(co2ppressures)

In [ ]:
t_couter = 0
for t in temperatures:
    pHs[:, t_couter] = np.array([equilibrate(t, p) for p in co2ppressures])
    t_couter += 1

In [ ]:
np.savetxt(results_folder + '/pHs.txt', pHs)

In [ ]:
import matplotlib.pyplot as plt
colors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C7', 'C8', 'C9']

In [ ]:
plt.figure()
for i in range(0, num_temperatures):
    plt.plot(co2ppressures, pHs[:, i], label=f'{temperatures[i]} C', color=colors[i])
plt.legend(loc="best")
plt.xlabel('ppCO2')
plt.ylabel('pH [-]')
plt.grid()
plt.savefig(results_folder + '/' + 'pH-vs-ppCO2.png', bbox_inches='tight')
plt.close()